In [2]:
!pip -q install pypdf faiss-cpu sentence-transformers transformers accelerate einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 34.0 MB/s eta 0:00:00


In [3]:
import os, re, math, textwrap
from typing import List, Dict, Tuple
from pypdf import PdfReader
import numpy as np
import faiss

from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Embeddings & generator
EMB_MODEL = "sentence-transformers/all-MiniLM-L6-v2"   # fast "finder brain"
GEN_MODEL = "google/flan-t5-base"                      # clear "writer brain"

embedder = SentenceTransformer(EMB_MODEL)
generator_tok = AutoTokenizer.from_pretrained(GEN_MODEL)
generator = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL)
gen = pipeline("text2text-generation", model=generator, tokenizer=generator_tok)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [4]:
from google.colab import files
up = files.upload()  # pick your PDF file
pdf_path = list(up.keys())[0]
pdf_path


Saving The Mountain Is You - Brianna Wiest.pdf to The Mountain Is You - Brianna Wiest.pdf


'The Mountain Is You - Brianna Wiest.pdf'

In [5]:
def load_pdf_pages(path:str):
    reader = PdfReader(path)
    pages = []
    for i, page in enumerate(reader.pages):
        try:
            txt = page.extract_text() or ""
        except:
            txt = ""
        # clean up spaces
        txt = re.sub(r"\s+", " ", txt).strip()
        if txt:
            pages.append({"page": i+1, "text": txt})
    return pages

pages = load_pdf_pages(pdf_path)
len(pages), [p["page"] for p in pages[:3]]


(239, [2, 3, 4])

In [6]:
def chunk_text(text:str, page:int, chunk_size=900, overlap=150):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunk = text[start:end]
        chunks.append({"page": page, "chunk": chunk})
        if end == len(text):
            break
        start = end - overlap
    return chunks

docs = []
for p in pages:
    docs += chunk_text(p["text"], p["page"])

len(docs), docs[0]["page"], docs[0]["chunk"][:150]


(465,
 2,
 'THE MOUNTAIN IS YOU TRANSFORMING SELF-SABOTAGE INTO SELF-MASTERY BRIANNA WIEST THOUGHTCATALOG.COM NEW YORK • LOS ANGELES')

In [7]:
def embed_texts(texts):
    vecs = embedder.encode(
        texts,
        convert_to_numpy=True,
        normalize_embeddings=True,
        show_progress_bar=True
    )
    return vecs.astype("float32")

corpus_texts = [d["chunk"] for d in docs]
embs = embed_texts(corpus_texts)

index = faiss.IndexFlatIP(embs.shape[1])  # cosine similarity
index.add(embs)
index.ntotal


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

465

In [8]:
def retrieve(query, k=5):
    qv = embed_texts([query])
    D, I = index.search(qv, k)
    ids = I[0].tolist()
    scores = D[0].tolist()
    return list(zip(ids, scores))

def format_context(hits):
    blocks = []
    for idx, score in hits:
        page = docs[idx]["page"]
        text = docs[idx]["chunk"]
        blocks.append(f"[p. {page}] {text}")
    return "\n\n".join(blocks)

# try a test search
hits = retrieve("definition of self-sabotage", k=3)
print(format_context(hits))


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[p. 27] 27THE MOUNTAIN IS YOU2 6 BRIANNA WIEST CHAPTER 2 THERE’S NO SUCH THING AS SELF-SABOTAGE WHEN YOU HABITUATE YOURSELF to do things that move your life forward, you call them skills. When they hold your life back, you call them self-sabotage. They are both essentially the same function. Sometimes, it happens by accident. Sometimes, we just get used to living a certain way and fail to have a vision for how life could be different. Sometimes, we make choices because we don’t know how to make better ones or that anything else is even possible. Sometimes, we settle for what we’re handed because we don’t know we can ask for more. Sometimes, we run our lives on autopilot for long enough that we begin to think we no longer have a choice. However, most of the time, it’s not accidental at all. The habits and behaviors you can’t stop engaging in—no mat- ter how destructive or limiting they may be—are

[p. 17] hese pre - existing beliefs, you begin to see how warped and illogical they were a

In [9]:
RAG_PROMPT = """You are a careful analyst.
You must ONLY use the CONTEXT to answer. If something is not in CONTEXT, say "Not in document."

Rules:
- Cite page numbers in every bullet using the [p. x] markers present in CONTEXT.
- Be concise and executive-friendly.

Task:
{task}

Output format:
- 3–5 bullet Executive Summary (each bullet ends with citation).
- 4–6 Key Findings (each bullet ends with citation).
- 2–4 Limitations (each bullet ends with citation if present; otherwise "Not in document").
- 3 Actionable Implications for Kenya (each bullet shows supporting citation).
- 2 Cross-Checks/Contradictions (if none, say "Not in document").

CONTEXT:
{context}
"""

def answer(task, k=6, max_new_tokens=450):
    hits = retrieve(task, k=k)
    ctx = format_context(hits)
    prompt = RAG_PROMPT.format(task=task, context=ctx)
    out = gen(prompt, max_new_tokens=max_new_tokens, do_sample=False)[0]["generated_text"]
    return out, ctx

# test run
task = "Define self-sabotage and give 3 implications for workplace performance in Kenya."
answer_text, used_context = answer(task, k=6)
print(answer_text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1486 > 512). Running this sequence through the model will result in indexing errors


Self-sabotage is a maladaptive coping mechanism, a way we give ourselves what we need without having to actually address what that need is. But like any coping mechanism, it is just that — a way to cope. It’s not an answer, it’s not a solution, and it does not ever truly solve the problem. We are merely numbing our de - sires, and giving ourselves a little taste of temporary relief. SELF-SABOTAGE COMES FROM IRRATIONAL FEAR Sometimes, our most sabotaging behaviors are really the result of long-held and unexamined fears we have about the world and ourselves. Perhaps it is the idea that you are unintelligent, unat - tractive, or disliked. Perhaps it is the idea of losing a job, [p. 11] 11THE MOUNTAIN IS YOU1 2 BRIANNA WIEST CHAPTER 1 THE MOUNTAIN IS YOU THERE IS NOTHING HOLDING you back in life more than yourself. If there is an ongoing gap between where you are and where you want to be—and your efforts to close it are consistently met with your own resistance, pain, and dis - comfort—sel

In [11]:
# --- helper: count tokens quickly ---
def count_tokens(text: str) -> int:
    return len(generator_tok(text, return_tensors="pt").input_ids[0])

# --- build a context that fits within a safe budget ---
def build_safe_context(query: str, k: int = 8, max_ctx_tokens: int = 350) -> str:
    # get extra hits, then trim down to fit
    hits = retrieve(query, k=k)
    blocks = []
    current = ""
    for idx, score in hits:
        page = docs[idx]["page"]
        text = docs[idx]["chunk"]
        block = f"[p. {page}] {text}\n\n"
        # try adding this block; stop if it would overflow
        if count_tokens(current + block) <= max_ctx_tokens:
            current += block
            blocks.append(block)
        else:
            break
    return current.strip()

RAG_PROMPT = """You are a careful analyst.
You must ONLY use the CONTEXT to answer. If something is not in CONTEXT, say "Not in document."

Rules:
- Cite page numbers in every bullet using the [p. x] markers present in CONTEXT.
- Be concise and executive-friendly.

Task:
{task}

Output format:
- 3–5 bullet Executive Summary (each bullet ends with citation).
- 4–6 Key Findings (each bullet ends with citation).
- 2–4 Limitations (each bullet ends with citation if present; otherwise "Not in document").
- 3 Actionable Implications for Kenya (each bullet shows supporting citation).
- 2 Cross-Checks/Contradictions (if none, say "Not in document").

CONTEXT:
{context}
"""

def answer_safe(task: str, k: int = 8, max_ctx_tokens: int = 350, max_new_tokens: int = 350):
    context = build_safe_context(task, k=k, max_ctx_tokens=max_ctx_tokens)
    prompt = RAG_PROMPT.format(task=task, context=context)
    # IMPORTANT: keep outputs modest too
    out = gen(
        prompt,
        max_new_tokens=max_new_tokens,
        do_sample=False
    )[0]["generated_text"]
    return out, context

# try again with the safer function
task = "Summarize the definition of self-sabotage in 3 bullet points with page citations."
answer_text, used_context = answer_safe(task, k=5, max_ctx_tokens=280, max_new_tokens=120)
print(answer_text)



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Self-sabotage.


In [12]:
GEN_MODEL = "google/flan-t5-large"  # bigger, smarter writer brain

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
generator_tok = AutoTokenizer.from_pretrained(GEN_MODEL)
generator = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL)
gen = pipeline("text2text-generation", model=generator, tokenizer=generator_tok)


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [13]:
task = "Summarize the definition of self-sabotage in 3 bullet points with page citations."
answer_text, used_context = answer_safe(task, k=5, max_ctx_tokens=280, max_new_tokens=120)
print(answer_text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

WHEN YOU HABITUATE YOURSELF to do things that move your life forward, you call them skills. When they hold your life back, you call them self-sabotage. They are both essentially the same function. Sometimes, it happens by accident. Sometimes, we just get used to living a certain way and fail to have a vision for how life could be different. Sometimes, we settle for what we’re handed because we don’t know how to make better ones or that anything else is even possible. Sometimes, we run our lives on auto


In [14]:
task = "Summarize the definition of self-sabotage in 3 bullet points with page citations."
answer_text, used_context = answer_safe(task, k=5, max_ctx_tokens=280, max_new_tokens=150)
print(answer_text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

WHEN YOU HABITUATE YOURSELF to do things that move your life forward, you call them skills. When they hold your life back, you call them self-sabotage. They are both essentially the same function. Sometimes, it happens by accident. Sometimes, we just get used to living a certain way and fail to have a vision for how life could be different. Sometimes, we settle for what we’re handed because we don’t know how to make better ones or that anything else is even possible. Sometimes, we run our lives on autopilot for long enough that we begin to think we no longer have a choice. However, most of the time, it’s not accidental at


In [15]:
task = "In 3 short bullet points with [p. x] citations, define what self-sabotage means according to the book."
answer_text, used_context = answer_safe(task, k=3, max_ctx_tokens=250, max_new_tokens=80)
print(answer_text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Self-sabotage. They are both essentially the same function. Sometimes, it happens by accident. Sometimes, we just get used to living a certain way and fail to have a vision for how life could be different. Sometimes, we settle for what we’re handed because we don’t know how to make better ones or that anything else is even possible. Sometimes, we


In [16]:
task = "List 4 symptoms of self-sabotage from the book with [p. x] citations."
answer_text, used_context = answer_safe(task, k=6, max_ctx_tokens=280, max_new_tokens=120)
print(answer_text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

3–5 bullet Executive Summary (each bullet ends with citation). - 4–6 Key Findings (each bullet ends with citation). - 2–4 Cross-Checks/Contradictions (if none, say "Not in document").


In [17]:
task = """
3–5 bullet Executive Summary (each bullet ends with citation).
4–6 Key Findings (each bullet ends with citation).
2–4 Cross-Checks/Contradictions (if none, say "Not in document").
"""
answer_text, used_context = answer_safe(task, k=6, max_ctx_tokens=280, max_new_tokens=180)
print(answer_text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Output: - Not in document


In [18]:
task = "Summarize what the book says about perfectionism in 3 bullets with [p. x] citations. Then give 2 contradictions if any."
answer_text, used_context = answer_safe(task, k=6, max_ctx_tokens=300, max_new_tokens=150)
print(answer_text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Not in document


In [19]:
GEN_MODEL = "facebook/bart-large-cnn"
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
generator_tok = AutoTokenizer.from_pretrained(GEN_MODEL)
generator = AutoModelForSeq2SeqLM.from_pretrained(GEN_MODEL)
gen = pipeline("summarization", model=generator, tokenizer=generator_tok)


config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Device set to use cpu


In [20]:
from textwrap import shorten
ctx = build_safe_context("perfectionism", k=5, max_ctx_tokens=400)
out = gen(ctx, max_length=120, min_length=60, do_sample=False)[0]['summary_text']
print(out)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

When we expect that our work must be perfect the first time we do it, we end up getting into a cycle of perfectionism. Perfectionism isn’t actually wanting everything to be right. In fact, it is a hindering thing, because it sets up unrealistic expectations about what we are capable of.


In [21]:
ctx = build_safe_context("perfectionism", k=5, max_ctx_tokens=400)
summary = gen(ctx, max_length=120, min_length=60, do_sample=False)[0]['summary_text']

print("### Executive Summary")
for bullet in summary.split(". "):
    if bullet.strip():
        print("- " + bullet.strip() + ".")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

### Executive Summary
- When we expect that our work must be perfect the first time we do it, we end up getting into a cycle of perfectionism.
- Perfectionism isn’t actually wanting everything to be right.
- In fact, it is a hindering thing, because it sets up unrealistic expectations about what we are capable of..
